# Brain MRI EDA

In [1]:
DATA_DIR_T2F = "/Users/eyash.p24/Desktop/MUJ/PBL5/PBL 5/Dataset/Brats/ASNR-MICCAI-BraTS2023-GLI-Challenge-Data/TrainingData/input_data_256/t2f/train"

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image

In [6]:
img_path = "/Users/eyash.p24/Desktop/MUJ/PBL5/PBL 5/Dataset/Brats/ASNR-MICCAI-BraTS2023-GLI-Challenge-Data/TrainingData/input_data_256/t2f/train/Image/image_BraTS-GLI-00000-000-t2f.jpeg"

In [7]:
img = Image.open(img_path)
img.format, img.mode, img.size

('JPEG', 'L', (180, 180))

In [8]:
np_file = "/Users/eyash.p24/Desktop/MUJ/PBL5/PBL 5/Dataset/Brats/ASNR-MICCAI-BraTS2023-GLI-Challenge-Data/TrainingData/input_data_4channels/Image/image_BraTS-GLI-00000-000.npy"

In [9]:
img2 = np.load(np_file)
img2.shape

(4, 180, 180)

In [ ]:
import nibabel as nib
import SimpleITK as sitk
import numpy as np

# Load NIfTI image
def load_nii_image(file_path):
    img = nib.load(file_path)
    return img.get_fdata(), img.affine  # Return image data and affine matrix

# Perform Skull Stripping
def skull_strip(input_image):
    # Convert the input image to SimpleITK image
    sitk_image = sitk.GetImageFromArray(input_image)
    
    # Use a simple Otsu thresholding for brain extraction
    otsu_filter = sitk.OtsuThresholdImageFilter()
    otsu_filter.SetInsideValue(1)
    otsu_filter.SetOutsideValue(0)
    binary_brain_mask = otsu_filter.Execute(sitk_image)
    
    # Convert the binary mask back to numpy array
    brain_mask = sitk.GetArrayFromImage(binary_brain_mask)
    
    # Apply the brain mask to the original image to remove non-brain regions
    skull_stripped_image = np.multiply(input_image, brain_mask)
    
    return skull_stripped_image, brain_mask

# Save the Skull-Stripped Image
def save_nii_image(output_path, skull_stripped_image, affine_matrix):
    stripped_img = nib.Nifti1Image(skull_stripped_image, affine_matrix)
    nib.save(stripped_img, output_path)
    print(f"Skull-stripped image saved to {output_path}")

# Main execution function
def main(input_nii_path, output_nii_path):
    # Step 1: Load the input MRI image
    image_data, affine = load_nii_image(input_nii_path)
    
    # Step 2: Perform skull stripping
    skull_stripped_data, brain_mask = skull_strip(image_data)
    
    # Step 3: Save the result
    save_nii_image(output_nii_path, skull_stripped_data, affine)

# Usage example
input_file_path = 'path/to/your/brain_mri.nii'  # Input NIfTI file
output_file_path = 'path/to/output/brain_mri_stripped.nii'  # Output NIfTI file
main(input_file_path, output_file_path)
